In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders, text2tokens
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, EncoderDecoderAttn, DecoderAttnRNN, \
    DecoderDotAttnRNN
from train_eval import count_parameters, summarize_results, plot_single_learning_curve, load_experiment_log
from train_eval import train_and_eval
import importlib
import pickle as pkl 
import torch

In [2]:
# model identification
MODEL_NAME = 'zh-seq2seq-rnn-dot-attn'
SRC_LANG = 'zh'
TARG_LANG = 'en'

# data processing params  
SRC_MAX_SENTENCE_LEN = 10
TARG_MAX_SENTENCE_LEN = 10
SRC_VOCAB_SIZE = 30000 #30000
TARG_VOCAB_SIZE = 30000 #30000

# model architecture params 
RNN_CELL_TYPE = 'gru'
NUM_LAYERS = 2 #2 
ENC_HIDDEN_DIM = 256 #512
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM #2 * ENC_HIDDEN_DIM 
TEACHER_FORCING_RATIO = 1
CLIP_GRAD_MAX_NORM = 1
ENC_DROPOUT = 0.2 # to actually implement
DEC_DROPOUT = 0.2 # to actually implement
USE_ATTN = True

# training params  
BATCH_SIZE = 64 #32
NUM_EPOCHS = 200
LR = 0.0003 # 0.0005
OPTIMIZER = 'Adam'
LAZY_TRAIN = False

In [3]:
# store as dict to save to results later 
params = {'model_name': MODEL_NAME, 'src_lang': SRC_LANG, 'targ_lang': TARG_LANG, 'rnn_cell_type': RNN_CELL_TYPE, 
          'src_max_sentence_len': SRC_MAX_SENTENCE_LEN, 'targ_max_sentence_len': TARG_MAX_SENTENCE_LEN, 
          'src_vocab_size': SRC_VOCAB_SIZE, 'targ_vocab_size': TARG_VOCAB_SIZE, 
          'num_layers': NUM_LAYERS, 'enc_hidden_dim': ENC_HIDDEN_DIM, 'dec_hidden_dim': DEC_HIDDEN_DIM,
          'teacher_forcing_ratio': TEACHER_FORCING_RATIO, 'clip_grad_max_norm': CLIP_GRAD_MAX_NORM,
          'enc_dropout': ENC_DROPOUT, 'dec_dropout': DEC_DROPOUT, 'use_attn': USE_ATTN, 
          'batch_size': BATCH_SIZE, 'num_epochs': NUM_EPOCHS, 'learning_rate': LR, 'optimizer': OPTIMIZER, 
          'lazy_train': LAZY_TRAIN} 

In [4]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [5]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, filter_long=False)
data_minibatch = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=BATCH_SIZE, filter_long=False) 
data_minitrain = process_data(SRC_LANG, TARG_LANG, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, vocab, sample_limit=1000, filter_long=False)

In [6]:
# create dataloaders 
loaders_full = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minibatch = create_dataloaders(data_minibatch, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
loaders_minitrain = create_dataloaders(data_minitrain, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [7]:
# define model 

encoder = EncoderRNN(rnn_cell_type=RNN_CELL_TYPE, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     src_max_sentence_len=SRC_MAX_SENTENCE_LEN, enc_dropout=ENC_DROPOUT, 
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))

# without attention 
decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS,
                     targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                     pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id'])

# with attention 
# decoder = DecoderAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
#                          num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
#                          targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
#                          pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']) 

# with dot attention 
decoder = DecoderDotAttnRNN(rnn_cell_type=RNN_CELL_TYPE, dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, 
                         num_layers=NUM_LAYERS, targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
                         targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, dec_dropout=DEC_DROPOUT, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoderAttn(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [ ]:
model, results = train_and_eval(
    model=model, loaders_full=loaders_full, loaders_minibatch=loaders_minibatch, loaders_minitrain=loaders_minitrain, 
    params=params, vocab=vocab, print_intermediate=100, save_checkpoint=True, save_to_log=True, 
    lazy_eval=True, print_attn=True, inspect_samples=1)

Epoch: 0.00, Train Loss: 0.00, Val Loss: 10.22, Train BLEU: 0.00, Val BLEU: 0.21, Minutes Elapsed: 0.08
Sampling from val predictions...
Source: 她 两年 年前 退休 了 结果 却 把 我家 变成
Reference: she retired two years ago , only to turn
Model: <SOS> the the the the the the the the the
Attention Weights: tensor([[0.0000, 0.0001, 0.0017, 0.0191, 0.2104, 0.1899, 0.4758, 0.1011, 0.0019,
         0.0000],
        [0.0015, 0.0039, 0.0185, 0.0621, 0.2173, 0.2102, 0.3286, 0.1397, 0.0169,
         0.0015],
        [0.0094, 0.0199, 0.0567, 0.1006, 0.1973, 0.1928, 0.2352, 0.1383, 0.0400,
         0.0098],
        [0.0248, 0.0425, 0.0912, 0.1195, 0.1787, 0.1703, 0.1828, 0.1198, 0.0500,
         0.0203],
        [0.0391, 0.0593, 0.1085, 0.1249, 0.1636, 0.1541, 0.1543, 0.1088, 0.0566,
         0.0307],
        [0.0516, 0.0710, 0.1146, 0.1248, 0.1560, 0.1464, 0.1409, 0.1005, 0.0579,
         0.0362],
        [0.0595, 0.0740, 0.1114, 0.1225, 0.1506, 0.1446, 0.1348, 0.0982, 0.0618,
         0.0426],
        [0.0670,

Epoch: 0.24, Train Loss: 0.00, Val Loss: 5.83, Train BLEU: 0.00, Val BLEU: 3.11, Minutes Elapsed: 12.51
Sampling from val predictions...
Source: 所以 如果 摄影 摄影师 就 在 这里 灯光 就 在
Reference: so if the photographer is right there and the
Model: <SOS> and we we we , we the the the
Attention Weights: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0009, 0.9196, 0.0001, 0.0794,
         0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.9846, 0.0000, 0.0000, 0.0151, 0.0000, 0.0000, 0.0000, 0.0003,
         0.0000],
        [0.0000,

Epoch: 0.42, Train Loss: 0.00, Val Loss: 5.60, Train BLEU: 0.00, Val BLEU: 2.77, Minutes Elapsed: 21.84
Sampling from val predictions...
Source: 在 加德 <UNK> 我 由 一些 从前 是 性 奴
Reference: in kathmandu , i was escorted by women who
Model: <SOS> and , , i i to , , ,
Attention Weights: tensor([[1.2489e-32, 3.7988e-28, 1.3232e-18, 2.2805e-29, 6.5388e-09, 2.1991e-11,
         2.5004e-12, 9.7186e-01, 2.8144e-02, 1.4819e-07],
        [9.8947e-02, 7.5292e-01, 5.5424e-05, 2.4275e-07, 1.4753e-01, 5.4556e-04,
         9.9324e-09, 2.7251e-07, 2.3719e-09, 2.3534e-13],
        [3.6037e-02, 9.6380e-01, 6.2407e-09, 1.6413e-04, 2.3395e-11, 2.5608e-16,
         9.1138e-20, 5.6749e-21, 5.1576e-23, 5.4442e-24],
        [2.8056e-15, 7.6525e-13, 1.1030e-15, 1.0000e+00, 1.0025e-18, 1.2645e-18,
         3.4176e-18, 1.4931e-22, 2.1754e-24, 1.2485e-24],
        [3.6802e-21, 2.1580e-20, 5.4749e-22, 1.0000e+00, 4.2186e-22, 1.1960e-22,
         4.5544e-20, 4.3155e-24, 1.4332e-27, 7.3037e-27],
        [1.5938e-14, 6.901

Epoch: 0.60, Train Loss: 0.00, Val Loss: 5.39, Train BLEU: 0.00, Val BLEU: 3.01, Minutes Elapsed: 31.20
Sampling from val predictions...
Source: 非常 非常感谢 感谢 肯尼 肯尼亚 尼亚 <UNK> <UNK> 在 我
Reference: <UNK> <UNK> . it means in my language ,
Model: <SOS> thank you you you you very . . .
Attention Weights: tensor([[3.9777e-26, 1.3082e-30, 2.6479e-36, 3.7835e-44, 5.8434e-43, 1.3236e-35,
         1.7014e-25, 1.4655e-19, 9.2228e-12, 1.0000e+00],
        [9.2496e-02, 9.0657e-01, 1.6101e-05, 1.0486e-18, 5.4863e-23, 2.9517e-22,
         1.8006e-22, 8.7288e-22, 2.2075e-15, 9.1755e-04],
        [5.9099e-07, 1.0000e+00, 1.1230e-06, 1.1959e-18, 1.9588e-21, 1.0768e-22,
         2.5299e-28, 2.5929e-31, 1.8534e-30, 2.0817e-17],
        [2.8664e-05, 9.9996e-01, 8.5684e-09, 1.8936e-20, 6.9622e-23, 1.0285e-21,
         1.9205e-23, 8.4594e-24, 1.1567e-17, 7.8809e-06],
        [7.8457e-05, 9.7864e-01, 4.8869e-08, 6.7727e-20, 1.1110e-22, 4.2182e-21,
         4.3976e-22, 2.5654e-21, 2.8240e-14, 2.1283e-02],
       

Epoch: 0.78, Train Loss: 0.00, Val Loss: 5.31, Train BLEU: 0.00, Val BLEU: 3.38, Minutes Elapsed: 40.56
Sampling from val predictions...
Source: 谁 还 会 认真 认真对待 真对 对待 他 如果 他
Reference: and who &apos;s going to take him seriously if
Model: <SOS> and &apos;s can &apos;t to to to to to
Attention Weights: tensor([[1.5600e-24, 1.6127e-37, 1.4973e-32, 5.6362e-25, 2.3097e-26, 3.7017e-17,
         1.3979e-12, 9.9999e-01, 1.7095e-08, 1.0064e-05],
        [1.0000e+00, 1.2666e-07, 9.6307e-12, 4.7812e-15, 3.2019e-25, 4.0484e-22,
         1.0508e-25, 1.7482e-14, 4.5859e-25, 1.3770e-16],
        [9.9883e-01, 1.1721e-03, 6.9983e-08, 2.8615e-10, 1.9603e-21, 5.6884e-19,
         2.1363e-28, 6.5282e-23, 3.2300e-31, 6.5071e-27],
        [9.9734e-01, 2.6500e-03, 4.4711e-06, 9.6505e-07, 1.0626e-14, 3.0761e-11,
         8.4428e-20, 1.6694e-14, 3.6160e-21, 1.1277e-19],
        [4.6390e-07, 1.6552e-04, 1.8426e-03, 9.9799e-01, 1.9106e-09, 8.8490e-09,
         2.2130e-15, 3.0598e-09, 5.4921e-13, 1.4139e-12],
    

...

Epoch: 9.60, Train Loss: 0.00, Val Loss: 4.36, Train BLEU: 0.00, Val BLEU: 6.42, Minutes Elapsed: 615.83
Sampling from val predictions...
Source: 我们 正 努力 与 当地 的 居民 们 沟通 <EOS>
Reference: we &apos;re working with local communities . <EOS> <PAD>
Model: <SOS> we &apos;re to to the people . <EOS> <EOS>
Attention Weights: tensor([[1.0000e+00, 6.0065e-30, 1.2959e-27, 4.8831e-33, 1.5166e-26, 8.4432e-22,
         7.7710e-16, 2.7102e-13, 2.0138e-09, 1.2937e-12],
        [1.0000e+00, 1.1801e-28, 4.4181e-25, 5.7538e-32, 1.5256e-33, 2.3595e-36,
         3.3787e-33, 3.5108e-34, 2.0888e-38, 4.2039e-45],
        [1.0000e+00, 2.6990e-18, 1.0897e-16, 9.9494e-18, 7.3239e-25, 1.4183e-29,
         2.3615e-30, 6.4846e-31, 4.3784e-38, 1.4013e-45],
        [6.9596e-19, 3.2003e-12, 8.1446e-02, 9.1855e-01, 8.6212e-06, 2.6711e-17,
         4.1501e-21, 6.8999e-27, 1.8868e-31, 4.0642e-39],
        [3.2905e-19, 1.0860e-15, 5.5364e-08, 9.9972e-01, 2.8124e-04, 1.0730e-17,
         7.0923e-23, 3.5985e-30, 1.0626e-38, 

Epoch: 9.78, Train Loss: 0.00, Val Loss: 4.38, Train BLEU: 0.00, Val BLEU: 6.43, Minutes Elapsed: 629.11
Sampling from val predictions...
Source: 我 想来 分享 几个 有用 的 方法 告诉 你们 如何
Reference: i want to share a few keys on how
Model: <SOS> i i a share you a of to telling
Attention Weights: tensor([[2.2508e-07, 2.2251e-20, 1.8204e-21, 2.7577e-22, 2.3683e-21, 1.6424e-19,
         1.6732e-15, 3.7668e-03, 9.9544e-01, 7.9007e-04],
        [1.0000e+00, 2.5122e-19, 8.9370e-19, 8.9572e-24, 6.7040e-31, 4.5973e-29,
         1.4605e-32, 1.7135e-27, 4.3531e-23, 2.7658e-28],
        [1.0000e+00, 5.6801e-09, 3.9001e-10, 5.2356e-16, 3.3217e-26, 5.7839e-34,
         9.3769e-38, 3.1032e-34, 1.0197e-33, 2.8821e-39],
        [1.9667e-26, 1.0567e-11, 9.9998e-01, 1.5864e-05, 2.3062e-15, 4.4990e-26,
         7.1893e-30, 4.4864e-30, 5.3666e-31, 1.0038e-32],
        [8.3548e-21, 7.1136e-08, 9.7332e-01, 2.6685e-02, 1.1883e-09, 1.7410e-15,
         6.4366e-19, 3.0840e-14, 3.7843e-19, 7.1054e-20],
        [6.1943e-22, 5

Epoch: 9.96, Train Loss: 0.00, Val Loss: 4.36, Train BLEU: 0.00, Val BLEU: 6.85, Minutes Elapsed: 642.37
Sampling from val predictions...
Source: 我 想 介绍 给 你们 认识 我 的 两个 兄弟
Reference: now i &apos;d like to introduce you to my
Model: <SOS> i want want to to introduce you you two
Attention Weights: tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.9565, 0.0425, 0.0010, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.4912, 0.5088, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.

Epoch: 10.15, Train Loss: 0.00, Val Loss: 4.40, Train BLEU: 0.00, Val BLEU: 6.78, Minutes Elapsed: 656.58
Sampling from val predictions...
Source: 这种 人 不存 存在 <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>
Reference: it doesn &apos;t exist . <EOS> <PAD> <PAD> <PAD>
Model: <SOS> and &apos;s &apos;t exist . <EOS> <EOS> <EOS> <EOS>
Attention Weights: tensor([[0.9860, 0.0000, 0.0000, 0.0000, 0.0140, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0001, 0.0000, 0.0001, 0.9998, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.9986, 0.0014, 0.0000, 0.0000, 0.0

Epoch: 10.36, Train Loss: 0.00, Val Loss: 4.41, Train BLEU: 0.00, Val BLEU: 6.56, Minutes Elapsed: 672.16
Sampling from val predictions...
Source: 希望 不 像 中间 那 张 那么 怪 <EOS> <PAD>
Reference: hopefully less awkward than that one in the middle
Model: <SOS> you don don the . . <EOS> <EOS> .
Attention Weights: tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0472, 0.3347, 0.6180, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0001, 0.7568, 0.2431, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3267, 0.0000, 0.6733, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0847, 0.0000, 0.9153, 0.0000, 0.0000,
         0.0000],
 

In [ ]:
summarize_results(load_experiment_log())[['dt_created', 'num_epochs', 'learning_rate', 'clip_grad_max_norm', 'val_loss']].head()

In [ ]:
plot_single_learning_curve(results)

In [ ]:
# Epoch: 199.00, Train Loss: 0.32, Val Loss: 13.19, Train BLEU: 98.94, Val BLEU: 0.27
plot_single_learning_curve(results)

In [ ]:
# with attention energies = v_broadcast.bmm(torch.tanh(self.attn(concat)).transpose(1, 2)) # switched order  
# Epoch: 199.00, Train Loss: 0.63, Val Loss: 12.82, Train BLEU: 92.05, Val BLEU: 0.38
plot_single_learning_curve(results)

In [ ]:
for i, token in enumerate(vocab[SRC_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
for i, token in enumerate(vocab[TARG_LANG]['id2token']): 
    if i < 20: 
        print("{}: {}".format(i, token))

In [ ]:
import torch
x = torch.arange(0, 3*5*10).view(3, 5, 10)
print(x)
y = x[1:, :, :]
print(y)
z = y.view(-1, 10)
print(z)

In [ ]:
t = torch.arange(0, 2*5).view(5, 2)
print(t)
u = t.contiguous().view(-1)
print(u)
v = t.permute(1, 0)
print(v)
w = v.contiguous().view(-1)
print(w)

In [ ]:
a = torch.arange(0, 2*1*300)
print(a)
b = a.view(-1, 1, 300)
print(b.size())

In [ ]:
for i, (src_idxs, targ_idxs, src_lens, targ_lens) in enumerate(loaders_full['train']):
#     print(i)
#     print(src_idxs.size())
#     print(src_idxs)
#     print(src_lens)
#     print(targ_idxs.size())
#     print(targ_idxs)
#     print(targ_lens)
    id2token = vocab[SRC_LANG]['id2token']
    test_tensor = src_idxs
    list_of_lists = test_tensor.numpy().astype(int).tolist()
    to_token = lambda l: ' '.join([id2token[idx] for idx in l])
    list_of_lists_tokens = [to_token(l) for l in list_of_lists] 
    break 

In [ ]:
#attn.data.masked_fill_(self.mask, -float('inf'))
test_tensor.

test_tensor.data.masked_fill_(test_tensor == 2, float('inf'))